In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

In [44]:
import findspark
findspark.init()

from pyspark import SparkConf,SparkContext
conf = SparkConf().setAppName("lrConf").setMaster('local')
sc = SparkContext(conf = conf)

from pyspark.sql import SparkSession
spark = SparkSession(sc).builder.appName('lrApp').getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=lrConf, master=local) created by __init__ at <ipython-input-2-7e6a0cadb6d3>:6 

In [45]:
df = spark.read.csv("./Resources/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Logistic_Regression/titanic.csv", inferSchema=True, header=True)

In [46]:
for i in df.head(5):
    print(i)
    print('\n')

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S')


Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C')


Row(PassengerId=3, Survived=1, Pclass=3, Name='Heikkinen, Miss. Laina', Sex='female', Age=26.0, SibSp=0, Parch=0, Ticket='STON/O2. 3101282', Fare=7.925, Cabin=None, Embarked='S')


Row(PassengerId=4, Survived=1, Pclass=1, Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age=35.0, SibSp=1, Parch=0, Ticket='113803', Fare=53.1, Cabin='C123', Embarked='S')


Row(PassengerId=5, Survived=0, Pclass=3, Name='Allen, Mr. William Henry', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='373450', Fare=8.05, Cabin=None, Embarked='S')




In [47]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [77]:
req_data = df.select(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']).na.drop()

In [78]:
from pyspark.ml.classification import LogisticRegression

In [79]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, VectorIndexer

In [80]:
gender_index = StringIndexer(inputCol='Sex', outputCol='SexIndex')

In [81]:
new_temp_gen_index = gender_index.fit(req_data).transform(req_data)

In [82]:
gen_one_encoded = OneHotEncoder(inputCol='SexIndex', outputCol='Gender')

In [83]:
new_temp_data = gen_one_encoded.transform(new_temp_gen_index)

In [84]:
new_temp_data.show()

+--------+------+------+----+-----+-----+-------+--------+--------+-------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|       Gender|
+--------+------+------+----+-----+-----+-------+--------+--------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|     0.0|(1,[0],[1.0])|
|       1|     1|female|38.0|    1|    0|71.2833|       C|     1.0|    (1,[],[])|
|       1|     3|female|26.0|    0|    0|  7.925|       S|     1.0|    (1,[],[])|
|       1|     1|female|35.0|    1|    0|   53.1|       S|     1.0|    (1,[],[])|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|     0.0|(1,[0],[1.0])|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|     0.0|(1,[0],[1.0])|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|     0.0|(1,[0],[1.0])|
|       1|     3|female|27.0|    0|    2|11.1333|       S|     1.0|    (1,[],[])|
|       1|     2|female|14.0|    1|    0|30.0708|       C|     1.0|    (1,[],[])|
|       1|     3

In [85]:
embark_index = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
new_temp_index_2 = embark_index.fit(new_temp_data).transform(new_temp_data)

embark_one_encoded = OneHotEncoder(inputCol='EmbarkIndex', outputCol='Embark')
new_req_data = embark_one_encoded.transform(new_temp_index_2)

In [86]:
new_req_data.show()

+--------+------+------+----+-----+-----+-------+--------+--------+-------------+-----------+-------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|       Gender|EmbarkIndex|       Embark|
+--------+------+------+----+-----+-----+-------+--------+--------+-------------+-----------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|     0.0|(1,[0],[1.0])|        0.0|(2,[0],[1.0])|
|       1|     1|female|38.0|    1|    0|71.2833|       C|     1.0|    (1,[],[])|        1.0|(2,[1],[1.0])|
|       1|     3|female|26.0|    0|    0|  7.925|       S|     1.0|    (1,[],[])|        0.0|(2,[0],[1.0])|
|       1|     1|female|35.0|    1|    0|   53.1|       S|     1.0|    (1,[],[])|        0.0|(2,[0],[1.0])|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|     0.0|(1,[0],[1.0])|        0.0|(2,[0],[1.0])|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|     0.0|(1,[0],[1.0])|        0.0|(2,[0],[1.0])|
|       0|     3|  male| 2.0

In [87]:
new_req_data = new_req_data.na.drop()

In [72]:
new_req_data.columns

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked',
 'SexIndex',
 'Gender',
 'EmbarkIndex',
 'Embark']

In [88]:
assembler = VectorAssembler(inputCols=['Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Gender',
 'Embark'], outputCol='features')

In [89]:
new_data = assembler.transform(new_req_data)

In [90]:
new_data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- SexIndex: double (nullable = false)
 |-- Gender: vector (nullable = true)
 |-- EmbarkIndex: double (nullable = false)
 |-- Embark: vector (nullable = true)
 |-- features: vector (nullable = true)



In [91]:
new_data.describe().show()

+-------+------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+------------------+------------------+
|summary|          Survived|            Pclass|   Sex|              Age|             SibSp|              Parch|              Fare|Embarked|          SexIndex|       EmbarkIndex|
+-------+------------------+------------------+------+-----------------+------------------+-------------------+------------------+--------+------------------+------------------+
|  count|               712|               712|   712|              712|               712|                712|               712|     712|               712|               712|
|   mean|0.4044943820224719| 2.240168539325843|  null|29.64209269662921|0.5140449438202247|0.43258426966292135| 34.56725140449432|    null|0.3637640449438202|0.2612359550561798|
| stddev|0.4911389472541192|0.8368543166903446|  null|14.49293290032352|0.9306921267673427| 0.8541814457454133

In [92]:
final_data = new_data.select(['features', 'Survived'])

In [93]:
final_data.show()

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[3.0,22.0,1.0,0.0...|       0|
|[1.0,38.0,1.0,0.0...|       1|
|(8,[0,1,4,6],[3.0...|       1|
|[1.0,35.0,1.0,0.0...|       1|
|[3.0,35.0,0.0,0.0...|       0|
|[1.0,54.0,0.0,0.0...|       0|
|[3.0,2.0,3.0,1.0,...|       0|
|[3.0,27.0,0.0,2.0...|       1|
|[2.0,14.0,1.0,0.0...|       1|
|[3.0,4.0,1.0,1.0,...|       1|
|(8,[0,1,4,6],[1.0...|       1|
|[3.0,20.0,0.0,0.0...|       0|
|[3.0,39.0,1.0,5.0...|       0|
|(8,[0,1,4,6],[3.0...|       0|
|(8,[0,1,4,6],[2.0...|       1|
|[3.0,2.0,4.0,1.0,...|       0|
|[3.0,31.0,1.0,0.0...|       0|
|[2.0,35.0,0.0,0.0...|       0|
|[2.0,34.0,0.0,0.0...|       1|
|(8,[0,1,4],[3.0,1...|       1|
+--------------------+--------+
only showing top 20 rows



In [94]:
log_reg = LogisticRegression(labelCol='Survived')

In [95]:
train, test = final_data.randomSplit([0.7,0.3])

In [96]:
lr_model = log_reg.fit(train)

In [97]:
test_res = lr_model.transform(test)

In [98]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [103]:
test_res.summary().show()

+-------+-------------------+-------------------+
|summary|           Survived|         prediction|
+-------+-------------------+-------------------+
|  count|                215|                215|
|   mean|0.42790697674418604|                0.4|
| stddev|0.49592996820348795|0.49104123587240905|
|    min|                  0|                0.0|
|    25%|                  0|                0.0|
|    50%|                  0|                0.0|
|    75%|                  1|                1.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



In [106]:
test_res.summary().prediction

Column<b'prediction'>

In [109]:
bin_eval = BinaryClassificationEvaluator(labelCol='Survived')

In [110]:
bin_eval.evaluate(test_res)

0.8562654648285616

In [111]:
test_res.show()

+--------------------+--------+--------------------+--------------------+----------+
|            features|Survived|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(8,[0,1,4],[3.0,1...|       1|[-0.7561282527237...|[0.31948745404363...|       1.0|
|(8,[0,1,4],[3.0,3...|       0|[-0.1082063169014...|[0.47297478459923...|       1.0|
|(8,[0,1,4,5],[3.0...|       0|[2.23784162798015...|[0.90359660669215...|       0.0|
|(8,[0,1,4,5],[3.0...|       0|[2.41654311737453...|[0.91808013130544...|       0.0|
|(8,[0,1,4,5],[3.0...|       0|[2.68461357340701...|[0.93611259783869...|       0.0|
|(8,[0,1,4,6],[1.0...|       1|[-3.4981439903129...|[0.02936508599852...|       1.0|
|(8,[0,1,4,6],[1.0...|       1|[-2.7094858289752...|[0.06241593399453...|       1.0|
|(8,[0,1,4,6],[1.0...|       1|[-1.8115688069870...|[0.14044862809436...|       1.0|
|(8,[0,1,4,6],[1.0...|       1|[-1.3673568323755...|[0.2030472256